In [ ]:
!pip -q install chromadb sentence-transformers transformers accelerate einops --upgrade

In [ ]:
import os
import time
import uuid
import json
from dataclasses import dataclass, asdict
from typing import List, Dict, Optional, Tuple

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

import chromadb
from chromadb.config import Settings

from sentence_transformers import SentenceTransformer

import textwrap

In [ ]:
CHAT_MODEL = "Qwen/Qwen2.5-0.5B-Instruct"
EMBED_MODEL = "all-MiniLM-L6-v2"
PERSIST_DIR = "/content/memory_db"

os.makedirs(PERSIST_DIR, exist_ok=True)
print("Persist dir:", PERSIST_DIR)

Persist dir: /content/memory_db


In [ ]:
class SBERTEmbeddings:
    def __init__(self, model_name: str = EMBED_MODEL):
        self.model = SentenceTransformer(model_name)

    def embed_texts(self, texts: List[str]) -> List[List[float]]:
        vecs = self.model.encode(
            texts, convert_to_numpy=True, normalize_embeddings=True)
        return [v.tolist() for v in vecs]

In [ ]:
client = chromadb.PersistentClient(
    path=PERSIST_DIR, settings=Settings(anonymized_telemetry=False))
collection = client.get_or_create_collection(
    name="chat_memory", metadata={"hnsw:space": "cosine"})

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(CHAT_MODEL)
model = AutoModelForCausalLM.from_pretrained(
    CHAT_MODEL,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    device_map="auto",
)
textgen = pipeline("text-generation", model=model, tokenizer=tokenizer)

Device set to use cpu


In [ ]:
def run_chat(messages: List[Dict[str, str]], max_new_tokens: int = 256, temperature: float = 0.2) -> str:
    prompt = tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True)
    out = textgen(prompt, max_new_tokens=max_new_tokens,
                  do_sample=True, temperature=temperature, top_p=0.9, top_k=50)
    gen = out[0]["generated_text"][len(prompt):]
    return gen.strip()

In [ ]:
@dataclass
class MemoryItem:
    id: str
    role: str
    content: str
    user_id: str
    conversation_id: str
    created_at: float

In [ ]:
def add_to_memory(items: List[MemoryItem]):
    texts = [i.content for i in items]
    embeddings = embedder.embed_texts(texts)
    collection.add(
        ids=[i.id for i in items],
        documents=texts,
        metadatas=[asdict(i) for i in items],
        embeddings=embeddings,
    )

In [ ]:
SYS_PROMPT = (
    "You are a concise research assistant. "
    "When helpful, cite retrieved notes inline as [memory n]. If memory seems irrelevant, say so briefly before answering."
)

In [ ]:
def build_context(retrieved, char_limit: int = 1600):
    chunks, used = [], 0
    for i, (doc, meta, dist) in enumerate(retrieved, start=1):
        s = f"[memory {i}] role={meta.get('role')} dist={dist:.3f} at {int(meta.get('created_at',0))}\n{doc.strip()}\n"
        if used + len(s) > char_limit:
            break
        chunks.append(s)
        used += len(s)
    return "\n".join(chunks) if chunks else "(no highly relevant memory found)"

In [ ]:
def answer(user_msg: str, top_k: int = 6) -> str:
    global CONV_ID
    retrieved = retrieve(user_msg, top_k=top_k, where={"user_id": USER_ID})
    context = build_context(retrieved)
    messages = [
        {"role": "system", "content": SYS_PROMPT},
        {"role": "user", "content": f"Use the MEMORY below only if helpful. Then answer the QUESTION.\n\nMEMORY:\n{context}\n\nQUESTION: {user_msg}"},
    ]
    reply = run_chat(messages)

    add_to_memory([
        MemoryItem(str(uuid.uuid4()), "user", user_msg,
                   USER_ID, CONV_ID, now_ts()),
        MemoryItem(str(uuid.uuid4()), "assistant",
                   reply, USER_ID, CONV_ID, now_ts()),
    ])
    return reply

In [ ]:
# ==== Global Variables & Utility Function ====

def now_ts():
    """Return current timestamp as float"""
    return time.time()

# Give unique IDs for this user and conversation
USER_ID = "user_" + str(uuid.uuid4())[:8]
CONV_ID = "conv_" + str(uuid.uuid4())[:8]

# Initialize the embedder (if not already done)
embedder = SBERTEmbeddings(EMBED_MODEL)

print("User ID:", USER_ID)
print("Conversation ID:", CONV_ID)


User ID: user_6cd26615
Conversation ID: conv_7fab7d19


In [ ]:
seed_items = [
    MemoryItem(str(uuid.uuid4()), "user", "My name is Geek.",
               USER_ID, CONV_ID, now_ts()),
    MemoryItem(str(uuid.uuid4()), "assistant",
               "Nice to meet you, Geek!", USER_ID, CONV_ID, now_ts()),
    MemoryItem(str(uuid.uuid4()), "user",
               "I prefer examples in Python, not JavaScript.", USER_ID, CONV_ID, now_ts()),
]
add_to_memory(seed_items)

In [ ]:
# =========================
# 🔁 Memory Functions Setup
# =========================

def add_to_memory(items: List[MemoryItem]):
    texts = [i.content for i in items]
    embeddings = embedder.embed_texts(texts)
    collection.add(
        ids=[i.id for i in items],
        documents=texts,
        metadatas=[asdict(i) for i in items],
        embeddings=embeddings,
    )

def retrieve(query: str, top_k: int = 6, where: Optional[Dict] = None):
    q_emb = embedder.embed_texts([query])[0]
    res = collection.query(
        query_embeddings=[q_emb],
        n_results=top_k,
        where=where,
        include=["documents", "metadatas", "distances"],
    )
    docs = res.get("documents", [[]])[0]
    metas = res.get("metadatas", [[]])[0]
    dists = res.get("distances", [[]])[0]
    return list(zip(docs, metas, dists))

def build_context(retrieved, char_limit: int = 1600):
    chunks, used = [], 0
    for i, (doc, meta, dist) in enumerate(retrieved, start=1):
        s = f"[memory {i}] role={meta.get('role')} dist={dist:.3f} at {int(meta.get('created_at',0))}\n{doc.strip()}\n"
        if used + len(s) > char_limit:
            break
        chunks.append(s)
        used += len(s)
    return "\n".join(chunks) if chunks else "(no highly relevant memory found)"

def answer(user_msg: str, top_k: int = 6) -> str:
    global CONV_ID
    retrieved = retrieve(user_msg, top_k=top_k, where={"user_id": USER_ID})
    context = build_context(retrieved)
    messages = [
        {"role": "system", "content": SYS_PROMPT},
        {"role": "user", "content": f"Use the MEMORY below only if helpful. Then answer the QUESTION.\n\nMEMORY:\n{context}\n\nQUESTION: {user_msg}"},
    ]
    reply = run_chat(messages)

    add_to_memory([
        MemoryItem(str(uuid.uuid4()), "user", user_msg,
                   USER_ID, CONV_ID, now_ts()),
        MemoryItem(str(uuid.uuid4()), "assistant",
                   reply, USER_ID, CONV_ID, now_ts()),
    ])
    return reply


In [ ]:
while True:
    user_msg = input("You: ")
    if user_msg.lower().strip() in ["exit", "quit", "stop"]:
        print("Exiting chat...")
        break
    reply = answer(user_msg)
    print("Assistant:", reply)

Assistant: AI stands for Artificial Intelligence, which refers to the simulation of human intelligence in machines that are programmed to think and learn like humans. AI involves developing algorithms and models that can perform tasks typically associated with humans, such as visual perception, speech recognition, decision-making, and problem-solving. It aims to create intelligent systems capable of learning from data, understanding natural language, and performing tasks autonomously or under certain conditions.
Assistant: The term "array" refers to a collection of elements that store related data values. Arrays are fundamental data structures used in programming languages to organize collections of items (such as numbers, strings, or other objects) into a sequence of elements. They allow developers to easily access and manipulate these elements by specifying their position within the array. This makes arrays versatile tools for various applications, including data storage, processing,